<a href="https://colab.research.google.com/github/MadhuryaPasan/Qwen3-ToolCalling-FineTune/blob/main/Notebook1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.3/432.3 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.5/376.5 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 95.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.9/181.9 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from unsloth import FastVisionModel
import torch

model, tokenizer = FastVisionModel.from_pretrained(
    "unsloth/Qwen3-VL-2B-Instruct-FP8",
    load_in_4bit = False,
    load_in_8bit = True,
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for long context
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [ ]:
# model = FastVisionModel.get_peft_model(
#     model,
#     finetune_vision_layers     = False, # False if not finetuning vision layers
#     finetune_language_layers   = True, # False if not finetuning language layers
#     finetune_attention_modules = True, # False if not finetuning attention layers
#     finetune_mlp_modules       = True, # False if not finetuning MLP layers

#     r = 8,
#     lora_alpha = 8,
#     lora_dropout = 0,
#     bias = "none",
#     random_state = 3407,
#     use_rslora = False,
#     loftq_config = None
# )
model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # False if not finetuning vision layers
    finetune_language_layers   = True, # False if not finetuning language layers
    finetune_attention_modules = True, # False if not finetuning attention layers
    finetune_mlp_modules       = True, # False if not finetuning MLP layers
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    r = 16,
    lora_alpha = 32,
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None
)

In [ ]:
from datasets import load_dataset
dataset = load_dataset("Salesforce/xlam-function-calling-60k", split = "train")

In [ ]:
dataset

In [ ]:
dataset[1]

In [ ]:
# @title
# def convert_xlam_to_conversation(sample):
#     # 1. Define the Standard xLAM System Instruction
#     system_instruction = (
#         "You are an expert in tool calling. You will be provided with a list of tools and a user query. Your goal is to output the correct tool call in JSON format."
#     )

#     # 2. Format the User prompt with clear delimiters
#     # Standard xLAM uses [BEGIN OF TOOL_DEFINITION] or similar structures
#     user_prompt = (
#         f"[BEGIN OF TOOL_DEFINITION]\n{sample['tools']}\n[END OF TOOL_DEFINITION]\n"
#         f"Query: {sample['query']}"
#     )

#     # 3. Format the Assistant output
#     # The 'answers' column in xLAM is already a JSON string
#     assistant_output = sample['answers']

#     return {
#         "messages": [
#             {"role": "system", "content": system_instruction},
#             {"role": "user", "content": user_prompt},
#             {"role": "assistant", "content": assistant_output}
#         ]
#     }

# converted_dataset = dataset.map(convert_xlam_to_conversation,remove_columns=dataset.column_names)

In [ ]:
# @title
# from unsloth.chat_templates import get_chat_template

# tokenizer = get_chat_template(
#     tokenizer,
#     chat_template = "qwen-3", # Qwen3 uses the same underlying ChatML structure
# )

# def formatting_prompts_func(sample):
#     # Standard xLAM setup for an Instruct model
#     messages = [
#         {"role": "system", "content": "You are a helpful assistant with access to tools. Output JSON tool calls."},
#         {"role": "user", "content": f"Tools:\n{sample['tools']}\n\nQuery: {sample['query']}"},
#         {"role": "assistant", "content": sample['answers']}
#     ]
#     return {"text": tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)}

# # Apply to your xLAM dataset
# converted_dataset = dataset.map(formatting_prompts_func)

In [ ]:
# converted_dataset[1]

In [ ]:
from unsloth.chat_templates import get_chat_template

# 1. Initialize the tokenizer with the correct Qwen3 template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen-3", # Use the built-in unsloth qwen-3 template
)

def formatting_prompts_func(sample):
    import json

    tools = json.loads(sample["tools"])
    ans_list = json.loads(sample["answers"])

    tool_calls = []
    for a in ans_list:
        tool_calls.append({
            "type": "function",
            "function": {
                "name": a["name"],
                "arguments": json.dumps(a["arguments"])  # <-- must be a string, not dict
            }
        })

    messages = [
        {"role": "user", "content": sample["query"]},
        {
            "role": "assistant",
            "content": "",
            "tool_calls": tool_calls
        }
    ]

    text = tokenizer.apply_chat_template(
        messages,
        tools=tools,
        tokenize=False,
        add_generation_prompt=False,
    )

    return {"text": text}

dataset = dataset.map(formatting_prompts_func, batched=False)

In [ ]:
dataset["text"][1]

In [ ]:
# from trl import SFTTrainer, SFTConfig

# FastVisionModel.for_training(model) # Enable for training!

# trainer = SFTTrainer(
#     model = model,
#     tokenizer = tokenizer,
#     train_dataset = dataset,
#     ataset_text_field = "text",
#     max_seq_length = 2048,
#     args = SFTConfig(
#         per_device_train_batch_size = 2,
#         gradient_accumulation_steps = 4,
#         warmup_steps = 5,
#         max_steps = 10,
#         # num_train_epochs = 1,
#         learning_rate = 5e-5,
#         fp16 = not torch.cuda.is_bf16_supported(),
#         bf16 = torch.cuda.is_bf16_supported(),
#         logging_steps = 1,
#         optim = "adamw_8bit",
#         weight_decay = 0.01,
#         lr_scheduler_type = "linear",
#         seed = 3407,
#         output_dir = "outputs",
#         report_to = "none",     # For Weights and Biases
#         remove_unused_columns = True,
#         packing = False
#     ),
# )
from trl import SFTTrainer, SFTConfig

FastVisionModel.for_training(model) # Enable for training!

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    args = SFTConfig(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 8,
        # warmup_steps = 5,
        warmup_ratio = 0.1,
        # max_steps = 10,
        num_train_epochs = 1,
        learning_rate = 5e-5,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.1,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",     # For Weights and Biases
        remove_unused_columns = True,
        packing = False
    ),
)

In [ ]:
trainer_stats = trainer.train()

In [ ]:
# 570

In [ ]:
!apt-get install zstd -y

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

In [ ]:
!nohup ollama serve > ollama.log 2>&1 &

In [ ]:
!apt-get install -y pciutils lshw

In [ ]:
!ollama --version

In [ ]:
# print(dataset[0])
# print(type(dataset[0]["text"]))

In [ ]:
from transformers import TrainingArguments
import json

# Test if training args are serializable
args_dict = trainer.args.to_dict()
try:
    json.dumps(args_dict)
    print("TrainingArguments OK")
except TypeError as e:
    print(f"Problem in TrainingArguments: {e}")
    for k, v in args_dict.items():
        try:
            json.dumps(v)
        except TypeError:
            print(f"  ❌ Non-serializable key: {k} = {v}")

In [ ]:
import json

config_dict = model.config.to_dict()

for k, v in config_dict.items():
    try:
        json.dumps(v)
    except TypeError:
        print(f"❌ Non-serializable key: {k} = {v}")

In [ ]:
print(type(tokenizer.chat_template))  # if this says <class 'function'>, that's your problem
print(tokenizer.chat_template)

In [ ]:
# !pip install unsloth[colab] --no-deps
# !pip install --upgrade --no-cache-dir unsloth


In [ ]:
# model.save_pretrained("lora_model") # Saves only adapter
# # Or for merging:
# model.save_pretrained_merged("model_final", tokenizer,)


In [ ]:
# model.save_pretrained_merged("output_dir", tokenizer, save_method="merged_16bit")

In [ ]:
# tokenizer.chat_template = tokenizer.chat_template
# tokenizer.save_pretrained("output_dir")

In [ ]:
# Remove BnB config artifact
del model.config.quantization_config

In [ ]:
model.save_pretrained_gguf("qwen_finetune", tokenizer,quantization_method = "q8_0",)

In [ ]:
model.push_to_hub_gguf("HF_USERNAME/llama_finetune", tokenizer, token = "...")

In [ ]:
import json
import os
from transformers import AutoConfig

# 1. Create a totally clean, static version of the config
# This removes all the "live" Python logic from Qwen3-VL
raw_config_dict = model.config.to_dict()

def make_serializable(d):
    """Deep-cleans a dictionary of anything that isn't simple data."""
    new_dict = {}
    for k, v in d.items():
        if isinstance(v, dict):
            new_dict[k] = make_serializable(v)
        elif isinstance(v, (str, int, float, bool, list)) or v is None:
            new_dict[k] = v
        else:
            # Skip functions, methods, and tensors
            continue
    return new_dict

clean_config_dict = make_serializable(raw_config_dict)

# 2. Inject the clean dict back into the model config
# We manually override the save function to use our clean data
def manual_save_json(path, use_diff=True):
    with open(path, "w") as f:
        json.dump(clean_config_dict, f, indent=2)

model.config.to_json_file = manual_save_json

# 3. Fix the chat template (the most common function culprit)
if hasattr(tokenizer, "chat_template"):
    tokenizer.chat_template = "<|im_start|>{{role}}\n{{content}}<|im_end|>\n"

# 4. Try saving again
print("🚀 Attempting forced local save...")
try:
    model.save_pretrained_merged("qwen3_final_merged", tokenizer, save_method = "merged_16bit")
    print("✅ SUCCESS! Your model is saved in the folder: qwen3_final_merged")
except Exception as e:
    print(f"❌ Failed again. Error: {e}")

In [ ]:
# from unsloth import FastLanguageModel

# # Load from the physical folder we just created
# model_gguf, tokenizer_gguf = FastLanguageModel.from_pretrained("qwen3_final_merged")

# model_gguf.push_to_hub_gguf(
#     "MadhuryaPasan/qwen3-2b-tools-on-steroid-v0",
#     tokenizer_gguf,
#     quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
#     token = ""
# )

In [ ]:
# Push the cleaned, merged model to HF in 16-bit first
model_gguf.push_to_hub(
    "MadhuryaPasan/qwen3-2b-tools-on-steroid-v0",
    token = "",
    save_method = "merged_16bit", # Important!
)
tokenizer_gguf.push_to_hub("MadhuryaPasan/qwen3-2b-tools-on-steroid-v0", token = "")